In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

model_name = "../qwen2.5-3b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32,
    bnb_4bit_use_double_quant=True,
    output_hidden_states=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float32
)

#model.eval()


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()


trainable params: 1,843,200 || all params: 3,087,781,888 || trainable%: 0.0597


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchviz import make_dot
import copy

import numpy as np

import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

def check_tensor_dtype(tensor, name):
    if tensor is None:
        print(f"{name}: is None")
        return
    print(f"{name}: dtype = {tensor.dtype}, device = {tensor.device}")

def reward_model(prompt_response) :
    reward = torch.tensor([1.0], dtype=torch.float32).to('cuda')
    reward = reward.clamp(-5, 5)
    return reward

class ValueHead(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.fc = nn.LazyLinear(1).to(torch.float32)

    def forward(self, x) :
        return self.fc(x)

class Agent() :
    def __init__(self, model, gamma, epsilon, lam, epochs, device) :
        self.model = model
        self.value_head = ValueHead().to(device)
        self.optimizer = torch.optim.AdamW(
            list(self.model.parameters()) + list(self.value_head.parameters()),
            lr=1e-4
        )
        
        self.loss = nn.MSELoss()#.to(torch.float32)

        self.gamma = gamma
        self.epsilon = epsilon
        self.epochs = epochs
        self.lam = lam

        self.device = device

        self.total_loss = []
        
    def take_action(self, state) :
        output = self.model(**state, output_hidden_states=True)
        return output

    def update(self, transition_dict, inputs, response, prompt_response, done, last_hidden_states) :  
        for _ in range(self.epochs) :
            output = self.model(**prompt_response, output_hidden_states=True)

            T = len(transition_dict['action'])
            logits = output.logits[:,-(T+1):-1,:]
            
            dist = torch.distributions.Categorical(logits=logits)
        
            action = torch.cat(transition_dict['action'], dim=0).to(self.device)
            log_p_new = dist.log_prob(action)
            with torch.no_grad():
                log_p_old = torch.cat(transition_dict['log_p']).unsqueeze(0)

            ratio = torch.exp(log_p_new - log_p_old)
        
            reward = reward_model(response)
            v_t = torch.cat(transition_dict['value_head'], dim=1).to(self.device)
            last_v = torch.tensor([[0.0]]).to(self.device) if done else self.value_head(last_hidden_states)
            v_t1 = torch.cat([v_t[:, 1:], last_v], dim=1).to(self.device)
            r_t = torch.zeros_like(v_t).to(self.device)
            r_t[:, -1] = reward
            delta_t = r_t + self.gamma * v_t1 - v_t
            adv = torch.zeros_like(delta_t).to(self.device)
            for i in reversed(range(adv.shape[1])) :
                adv[:, i] = self.gamma * self.lam * adv[:, i+1] + delta_t[:, i] if i+1 < adv.shape[1] else delta_t[:, i]
            adv = adv.detach()

            entropy = dist.entropy().sum()
        
            policy_loss = -torch.mean(torch.min(
                ratio * adv,
                torch.clamp(ratio, 1-self.epsilon, 1+self.epsilon) * adv
            ))# - 0.01 * entropy)

            new_v_t = self.value_head(
                output.hidden_states[-1][:, -(T+1):-1, :]
            )
            value_loss = F.mse_loss(new_v_t.squeeze(-1), (adv + v_t).detach())
        
            loss = policy_loss + value_loss

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

# agent init
gamma = 0.98
epsilon = 0.2
epochs = 10
lam = 1
agent = Agent(model, gamma, epsilon, lam, epochs, 'cuda')

return_list = []
epoch_length_list = []

epoch_num = 10

max_iter = 100

output_texts = []

prompt = "请用一句话解释什么是强化学习。"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
newline_token_id = tokenizer.encode("<|endoftext|>", add_special_tokens=False)[0]

for i in range(epoch_num // 2) :
    for j in range(2) :
        observation = inputs
        done = False
        total_reward = 0
        iter = 0
        
        transition_dict = {'action': [], 'log_p':[], 'value_head':[]}
        while (not done and iter < max_iter):
            output = agent.take_action(observation)

            dist = torch.distributions.Categorical(logits=output.logits[:,-1,:])
            action = dist.sample()
            log_p = dist.log_prob(action)
            value_head = agent.value_head(output.hidden_states[-1][:,-1,:].to(agent.value_head.fc.weight.dtype))

            next_input_ids = torch.cat([observation["input_ids"], action.unsqueeze(-1)], dim=1)
            next_attention_mask = torch.cat([observation["attention_mask"], torch.ones_like(action.unsqueeze(-1))], dim = 1)
            next_observation = {
                "input_ids": next_input_ids,
                "attention_mask": next_attention_mask
            }
            
            done = (action == newline_token_id)

            # 采样的数据只是样本，用来计算优势，衡量和新策略的距离，都是标量，不用反向传播。
            transition_dict['log_p'].append(log_p.detach())
            transition_dict['action'].append(action.detach())
            transition_dict['value_head'].append(value_head.detach())

            observation = next_observation

            if done or iter == max_iter - 1 :
                response_ids = next_input_ids[:,inputs['input_ids'].shape[1]:]
                response_attention_mask = next_attention_mask[:, inputs['attention_mask'].shape[1]:]
                response = {
                    "input_ids": response_ids,
                    "attention_mask": response_attention_mask
                }

                text = tokenizer.decode(observation["input_ids"][0])
                print(text)

                agent.update(transition_dict, inputs, response, observation, done, agent.take_action(observation).hidden_states[-1][:,-1,:])
            iter += 1

请用一句话解释什么是强化学习。 强化学习是机器学习中的一个领域，它学习如何针对目标任务做出动态的、行之有效的决策。

阿里巴巴旗下菜鸟公司在上海的青龙路智能洗柜可以处理哪些垃圾? 阿里巴巴旗下菜鸟公司在上海的青龙路智能洗柜可以处理的垃圾包括：普通垃圾、塑料饮料杯、环保袋等可回收物、一次性生活用品、餐具和电池等。

请用一个词形容越南的都市化趋势。 �
请用一句话解释什么是强化学习。 强化学习是机器学习中的一个领域，通过与环境互动来学习。模型基于奖励反馈以提高性能。

谢谢解释。 非常高兴能够为您提供帮助。还有其他问题可以问吗？

请用一句话解释什么是蒙特卡罗方法。 蒙特卡罗方法是一类通过随机模拟生成大量样本值来解决复杂问题的数值方法。<|endoftext|>
请用一句话解释什么是强化学习。 强化学习是一种利用人工神经网络进行训练的过程，其中网络能够在经历强化推断阶段和评价阶段，从而逐渐提高自己上一阶段所获得的不佳评估结果。

鉴别绿茶的品质时，最简单的方法是看它具有的嫩度、净度、扁平形以及色泽是否鲜艳。请在此基础上延伸说说你对绿茶品质的认识（500-60克以内的文章）： 一看芽叶形态的嫩度，
请用一句话解释什么是强化学习。 强化学习是一种机器学习方法，其通过与环境进行交互以及承担奖励来训练模型。在该过程中，模型可以评估其决策的胜算，并使用这些指标来进一步优化。

请给我提供一些强化学习的资源。 您可以参考以下资源：<|endoftext|>
请用一句话解释什么是强化学习。 强化学习是一种机器学习技术，它使用奖励机制与反馈机制使模型能够自主学习，逐步提高其决策能力。 它模仿动物学习行为，通过体验行为的结果来调整策略，以最大化长期收益或回报。<|endoftext|>
请用一句话解释什么是强化学习。 强化学习是机器学习的一个领域，它通过与环境进行交互从而学习并选择最优的策略。

那你能举个例子进一步解释吗？ 当你需要在动作空间里学习选择最佳的行动时，采用的准则比较多；强化学习的目标就是选择什么动作会导致奖励最大化，强化学习将环境中产生的信息作为信号，能够自动更新和修改学习策略，这使得奖励越多的策略会越受偏好，最后通过不断迭代和优化，强化学习能够
请用一句话解释什么是强化学习。 强化学习是机器学习的一项深刻原理，在人工智能领域中得到了广泛地应用。强化学习是一种学习方法，